In [1]:
'''
Created on 2018. 2. 14.

@author: phs

1. 대화 말뭉치 파일을 읽어들인다.
2. 딥러닝(tensorflow) 모델 생성에 사용한 자료구조를 읽어 들여서 반환한다.
3. 딥러닝(tensorflow)으로 생성한 자연어 이해 모델을 읽어들여서 반환한다.
4. 입력받은 문장을 자연어 처리한다(토크나이저, 스태밍).
5. 자연어 처리한 문장을 Bag of word 생성하여 반환한다.
6. 입력받은 문장을 학습 모델(자연어 이해 모델)을 이용 분류하여  결과를 반환한다.
7. 분류된 말뭉치 대화에서 임의로 한 문장을 선택하여 입력받은 문장의 대답으로 반혼한다.

'''
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# things we need for NLP
import json
import pickle
import random

# import nltk
# from nltk.stem.lancaster import LancasterStemmer
# stemmer = LancasterStemmer()

# from konlpy.tag import Komoran
from konlpy.tag import Okt
# komoran = Komoran()
twitter = Okt()


import tflearn
import tensorflow as tf
import numpy as np
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

C:\Users\saint\Anaconda3\envs\chat_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\saint\Anaconda3\envs\chat_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\saint\Anaconda3\envs\chat_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\saint\Anaconda3\envs\chat_env\lib\site-packages\tensorf

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:
# import our chat-bot intents file
def read_dialog_intents_jsonfile(input_file_name):
    """
     대화 말뭉치 파일을 읽어들인다.
    """
    with open(input_file_name, 'rt', encoding='UTF8') as json_data:
        intents = json.load(json_data)
        
    return intents

# 대화 말뭉치와 대화 의도가 정의된 JSON 문서 집합 읽기
input_file_name = './DialogIntents/intents_home_kr.json'
intents = read_dialog_intents_jsonfile(input_file_name)


In [3]:
# restore all of our data structures
def restore_training_data_structures(input_training_data_file_name):
    """
     딥러닝(tensorflow) 모델 생성에 사용한 자료구조를 읽어 들여서 반환한다.
    """
    # restore all of our data structures
    data = pickle.load( open( input_training_data_file_name, "rb" ) )
    words = data['words']
    classes = data['classes']
    train_x = data['train_x']
    train_y = data['train_y']
    
    return classes, words, train_x, train_y

#  딥러닝(tensorflow) 모델 생성에 사용한 자료구조를 읽어들임
input_training_data_file_name = "./NLUModel/training_data_home_kr"
classes, words, train_x, train_y = restore_training_data_structures(input_training_data_file_name)


In [4]:
# restore all of trained tflearn model file
def restore_training_model(train_x, train_y, tflearn_logs_dir):
    """
     딥러닝(tensorflow)으로 생성한 자연어 이해 모델을 읽어들여서 반환한다.
    """
    # Build neural network
    net = tflearn.input_data(shape=[None, len(train_x[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
    net = tflearn.regression(net)
    
    # Define model and setup tensorboard
    model = tflearn.DNN(net, tensorboard_dir=tflearn_logs_dir)

    return model

#  딥러닝(tensorflow)으로 생성한 자연어 이해 모델을 읽어들임
tflearn_logs_dir = "home_tflearn_kr_logs"
model = restore_training_model(train_x, train_y, tflearn_logs_dir)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [5]:
# restore all of trained tflearn model file
def restore_training_model(train_x, train_y, tflearn_logs_dir):
    """
     딥러닝(tensorflow)으로 생성한 자연어 이해 모델을 읽어들여서 반환한다.
    """
    # Build neural network
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(8, input_shape=(len(train_x[0]))),
        tf.keras.layers.Dense(8),
        tf.keras.layers.Dense(len(train_y[0]), activation="softmax"),
        ])
    
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics="accuracy")
    model.fit(train_x, train_y, epochs=1000, batch_size=8)

In [6]:
# load our saved model
tflearn_model_file_name = "./NLUModel/model_home_kr.tflearn"
model.load(tflearn_model_file_name)


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\saint\GroupProject\step4\jupyter_app\nlp\home_chat\NLUModel\model_home_kr.tflearn


In [17]:
# things we need for Tensorflow

# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25

def clean_up_sentence(sentence):
    # tokenize the pattern
#     sentence_words = nltk.word_tokenize(sentence)
    pos_result = twitter.pos(sentence, norm=True, stem=True)
#     print("sentence pos_result[%s]" % pos_result)
#     print("sentence pos_result[0][0][%s]" % pos_result[0][0])
#     print("sentence pos_result[1][0][%s]" % pos_result[1][0])
#     print("sentence pos_result[][%s]" % pos_result[0])
    sentence_words = [lex for lex, pos in pos_result]
    # stem each word
#     sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


def classify(sentence, words, classes, model):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, intents, words, classes, model, userID='123', show_details=False):
    results = classify(sentence, words, classes, model)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return random.choice(i['responses'])

            results.pop(0)


In [11]:
sentence = '안녕?'
print("classify('안녕?')[{}]".format(classify(sentence, words, classes, model)))
print("response('안녕?') ==> [{}]".format(response(sentence, intents, words, classes, model)))
print()


classify('안녕?')[[('greeting', 0.99066764)]]
response('안녕?') ==> [다시 만나서 반가워요.]



In [18]:
context

{}

In [20]:
sentence = '무슨 제품이 있나요?'
print("classify('무슨 제품이 있나요?')[%s]" % classify(sentence, words, classes, model))
print("response('무슨 제품이 있나요?') ==> [%s]" % response(sentence, intents, words, classes, model))
print()

classify('무슨 제품이 있나요?')[[('product', 0.999616)]]
response('무슨 제품이 있나요?') ==> [채팅 대화하는 홈페이지, 채팅 대화 고객 FAQ, 채팅 대화 고객 문의 자동 응답 시스템을 클라우드 서비스와 패키지 기반 구축 서비스로 제공합니다.]



In [21]:
# show context
context

{'123': 'serviceTechnology'}

In [22]:
sentence = '구체적인 기술은?'
print("classify('구체적인 기술은?')[%s]" % classify(sentence, words, classes, model))
print("response('구체적인 기술은?') ==> [%s]" % response(sentence, intents, words, classes, model))
print()

classify('구체적인 기술은?')[[('possessTechnology', 0.9998584)]]
response('구체적인 기술은?') ==> [부트스트랩 (CSS 프레임워크),jQuery,머신러닝, TensorFlow,자연어처리,자연어이해,Python 등 입니다.]



In [13]:
# show context
context

{'123': 'serviceTechnology'}

In [14]:
sentence = '제품기술'
print("classify('제품기술')[{}]".format(classify(sentence, words, classes, model)))
print("response('제품기술') ==> [{}]".format(response(sentence, intents, words, classes, model)))
print()

classify('제품기술')[[('possessTechnology', 0.95949155)]]
response('제품기술') ==> [부트스트랩 (CSS 프레임워크),jQuery,머신러닝, TensorFlow,자연어처리,자연어이해,Python 등 입니다.]



In [15]:
# clear context
#response("Hi there!", show_details=True)
sentence = '안녕'
print("response('안녕?') ==> [{}]".format(response(sentence, intents, words, classes, model,show_details=True)))
print("classify('안녕?')[{}]".format(classify(sentence, words, classes, model)))
print()

context: 
tag: greeting
response('안녕?') ==> [안녕하세요, 어떻게 도와 드릴까요?]
classify('안녕?')[[('greeting', 0.9875065)]]



In [16]:
sentence = '서비스기술'
print("classify('서비스기술')[{}]".format(classify(sentence, words, classes, model)))
print("response('서비스기술') ==> [{}]".format(response(sentence, intents, words, classes, model)))
print()

classify('서비스기술')[[('possessTechnology', 0.90978044)]]
response('서비스기술') ==> [None]



In [17]:
# show context
context

{'123': ''}

In [18]:
sentence = '안녕히 계셔요.'
print("classify('안녕히 계셔요.')[{}]".format(classify(sentence, words, classes, model)))
print("response('안녕히 계셔요.') ==> [{}]".format(response(sentence, intents, words, classes, model)))
print()

classify('안녕히 계셔요.')[[('goodbye', 0.9815683)]]
response('안녕히 계셔요.') ==> [안녕히 가세요]



In [19]:
# show context
context

{'123': ''}

In [20]:
sentence = '뭐야?'
print("classify('뭐야?')[{}]".format(classify(sentence, words, classes, model)))
print("response('뭐야?') ==> [{}]".format(response(sentence, intents, words, classes, model)))
print()

classify('뭐야?')[[('greeting', 0.34372285), ('Slang', 0.3057799)]]
response('뭐야?') ==> [다시 만나서 반가워요.]

